In [1]:
dt = 1

1. Delivery
2. Crane Operation Time - not being ready 8- 10
3. Every unit cost 5000 $ - more than 4 worker
4. only 4 types of pbu - 5, 10, 15 

Equations

$ Y_{warehouse}(t) =  Y_{warehouse}(t-1) + x_{delivered}(t) - x_{load}(t) $


$ Y_{parking}(t) =  Y_{parking}(t-1) + x_{load}(t) - x_{hoist}(t) $


$ Y_{Hoist}(t) =  Y_{Hoist}(t-1) + x_{hoist}(t) - x_{install}(t) $


$ Y_{install}(t) =  Y_{install}(t-1) + x_{install}(t) $


$Y_{parked}(t)$ =< 30


In [4]:
import pandas as pd 
import datetime
import numpy as np
from ortools.linear_solver import pywraplp


## Setting up the optimization

In [5]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')
 
 

## Generate Random delivery Data

In [6]:
delivery=pd.DataFrame([])
delivery["date"]=pd.date_range(start='1/1/2022', end='3/31/2022')
delivery["shipment"]=np.random.poisson(10, len(delivery))
delivery["time_index"]=range(1,len(delivery)+1)
n=len(delivery)

In [7]:
delivery.head()

,date,shipment,time_index
0,2022-01-01,8,1
1,2022-01-02,8,2
2,2022-01-03,14,3
3,2022-01-04,7,4
4,2022-01-05,6,5


## Defining Variables

$x_{delivered}(t)$     $~~~~~~~~~~~~~~~~$     PBUs delivered into the warehouse at each time frame

$Y_{warehouse}(t)$     $~~~~~~~~~~~~~~~~$     PBUs in warehouse at each time frame

$x_{load}(t)$     $~~~~~~~~~~~~~~~~$     PBUs loaded from the warehouse into the parking at each time frame

$Y_{parked}(t)$     $~~~~~~~~~~~~~~~~$     PBUs in parking at each time frame

$x_{hoist}(t)$     $~~~~~~~~~~~~~~~~$     PBUs selected from the parking, and and to be hoisted at each time frame

$Y_{hoisted}(t)$     $~~~~~~~~~~~~~~~~$     PBUs which are hoisted at each time frame

$x_{install}(t)$     $~~~~~~~~~~~~~~~~$     PBUs selected from the hoisted ones, and to be installed at each time frame

$Y_{installed}(t)$     $~~~~~~~~~~~~~~~~$     PBUs installed at each time frame



$C_{hoist}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs can be hoisted at each time frame

$C_{install}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs can be installed at each time frame

$C_{parked}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs in parking at each time frame

$C_{load}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs loaded from the warehouse into the parking at each time frame

$C_{installed}$     $~~~~~~~~~~~~~~~~$   Maximum  PBUs installed 





In [8]:
C_hoist= 10
C_install= 8
C_parked= 30
C_load=50
C_installed=44

In [9]:
Initial_Warehouse_Quantity=0

In [10]:
infinity = solver.infinity()
Y_warehouse={}

for i in range (0,n):
    Y_warehouse[i]=solver.IntVar(lb=0,ub=infinity,name="Y_warehouse_%i" % i)

In [11]:
Y_parked={}

for i in range (0,n):
    Y_parked[i]=solver.IntVar(lb=0,ub=C_parked,name="Y_parked_%i" % i)

In [12]:
x_load={}

for i in range (0,n):
    x_load[i]=solver.IntVar(lb=0,ub=C_load,name="x_load_%i" % i)

In [13]:
x_hoist={}
for i in range (0,n):
    x_hoist[i]=solver.IntVar(lb=0,ub=C_hoist,name="x_hoist_%i" % i)

In [14]:
Y_hoisted={}
for i in range (0,n):
    Y_hoisted[i]=solver.IntVar(lb=0,ub=C_hoist,name="Y_hoisted_%i" % i)

In [15]:
x_install={}
for i in range (0,n):
    x_install[i]=solver.IntVar(lb=0,ub=C_install,name="x_install_%i" % i)

In [16]:
Y_installed={}
for i in range (0,n):
    Y_installed[i]=solver.IntVar(lb=0,ub=C_installed,name="Y_installed_%i" % i)

In [21]:
print('Number of variables =', solver.NumVariables())


Number of variables = 630


## Defining Mathematical Relationship Between Variables 

The number of the PBUs in the warehouse at time $t$ = The number of the PBUs in the warehouse at time $t-1$  +  the number of PBUs delivered to warehouse at time $t$   -  the number of PBUs loaded from warehouse to parking at time $t$

$ Y_{warehouse}(t) =  Y_{warehouse}(t-1) + x_{delivered}(t) - x_{load}(t) $


In [23]:
for i in range(1,n):
    solver.Add(Y_warehouse[t]== Y_warehouse[t-1] + delivery["shipment"].values[t] - x_load[t])


The number of PBUs in parking at time $t$ = The number of PBUs in parking at time $t-1$ + The number of PBUs loaded from warehouse to parking at time $t$ - The number of PBUs selected from the parking, and to be hoisted at time $t$

$ Y_{parked}(t) =  Y_{parked}(t-1) + x_{load}(t) - x_{hoist}(t) $


In [50]:
for i in range(1,n):
    solver.Add(Y_parked[t]== Y_parked[t-1] + x_load[t] - x_hoist[t])

In [51]:
Initial_Warehouse_Quantity=100

In [53]:
solver.Add(Y_warehouse[0] ==Initial_Warehouse_Quantity)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000025D7D8C2870> >

In [54]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 181


In [63]:
solver.Minimize(Y_warehouse[89])

solver.Solve()



0

In [64]:
solver.Objective().Value()

0.0

In [65]:
solver.OPTIMAL

0

In [71]:
Y_warehouse

{0: Y_warehouse_0,
 1: Y_warehouse_1,
 2: Y_warehouse_2,
 3: Y_warehouse_3,
 4: Y_warehouse_4,
 5: Y_warehouse_5,
 6: Y_warehouse_6,
 7: Y_warehouse_7,
 8: Y_warehouse_8,
 9: Y_warehouse_9,
 10: Y_warehouse_10,
 11: Y_warehouse_11,
 12: Y_warehouse_12,
 13: Y_warehouse_13,
 14: Y_warehouse_14,
 15: Y_warehouse_15,
 16: Y_warehouse_16,
 17: Y_warehouse_17,
 18: Y_warehouse_18,
 19: Y_warehouse_19,
 20: Y_warehouse_20,
 21: Y_warehouse_21,
 22: Y_warehouse_22,
 23: Y_warehouse_23,
 24: Y_warehouse_24,
 25: Y_warehouse_25,
 26: Y_warehouse_26,
 27: Y_warehouse_27,
 28: Y_warehouse_28,
 29: Y_warehouse_29,
 30: Y_warehouse_30,
 31: Y_warehouse_31,
 32: Y_warehouse_32,
 33: Y_warehouse_33,
 34: Y_warehouse_34,
 35: Y_warehouse_35,
 36: Y_warehouse_36,
 37: Y_warehouse_37,
 38: Y_warehouse_38,
 39: Y_warehouse_39,
 40: Y_warehouse_40,
 41: Y_warehouse_41,
 42: Y_warehouse_42,
 43: Y_warehouse_43,
 44: Y_warehouse_44,
 45: Y_warehouse_45,
 46: Y_warehouse_46,
 47: Y_warehouse_47,
 48: Y_wareh

Y_parked

In [68]:
Y_warehouse_solution=[Y_warehouse[i].solution_value() for i in range(0,n)]
x_load_solution=[x_load[i].solution_value() for i in range(0,n)]
Y_parked_solution=[Y_parked[i].solution_value() for i in range(0,n)]
x_hoist_solution=[x_hoist[i].solution_value() for i in range(0,n)]
Y_hoisted_solution=[Y_hoisted[i].solution_value() for i in range(0,n)]
x_install_solution=[x_install[i].solution_value() for i in range(0,n)]
Y_installed_solution=[Y_installed[i].solution_value() for i in range(0,n)]
 
 

AttributeError: 'float' object has no attribute 'solution_value'

In [ ]:
      
$Y_{hoisted}(t)$     
$x_{install}(t)$     
$Y_{installed}(t)$

In [ ]:
solver.Add(x + 2 * y <= 14.0)